# Spam/Ham classification Model

In [1]:
# read file into pandas using a relative path
import pandas as pd
path = 'data/spam.tsv'
df = pd.read_table(path, header=None, names=['label', 'message'])

In [2]:
# examine the shape
df.shape

(5572, 2)

In [3]:
# examine the first 10 rows
df.head(10)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [4]:
# examine the class distribution
df.label.value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: label, dtype: float64

In [5]:
# convert label to a numerical variable
df['label_num'] = df.label.map({'ham':0, 'spam':1})

In [6]:
# check that the conversion worked
df.head(10)

,label,message,label_num
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
6,ham,Even my brother is not like to speak with me. ...,0
7,ham,As per your request 'Melle Melle (Oru Minnamin...,0
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1


In [7]:
# how to define X and y (from the SMS data) for use with COUNTVECTORIZER
X = df.message
y = df.label_num
print(X.shape)
print(y.shape)

(5572,)
(5572,)


Notice that X is one dimensional here. When we pass it to count vectorizer, it will convert it to two dimensional matrix. 

In [8]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4179,)
(1393,)
(4179,)
(1393,)


In [9]:
# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [10]:
X_train_dtm = cv.fit_transform(X_train)

In [11]:
# examine the document-term matrix
X_train_dtm

<4179x7456 sparse matrix of type '<class 'numpy.int64'>'
	with 55209 stored elements in Compressed Sparse Row format>

In [12]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = cv.transform(X_test)
X_test_dtm

<1393x7456 sparse matrix of type '<class 'numpy.int64'>'
	with 17604 stored elements in Compressed Sparse Row format>

## Multinomial Naive Bayes

In [13]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [14]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
nb.fit(X_train_dtm, y_train)

MultinomialNB()

In [15]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [16]:
# calculate accuracy of class predictions
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
accuracy_score(y_test, y_pred_class)

0.9885139985642498

In [17]:
nb.classes_

array([0, 1], dtype=int64)

In [18]:
# print the confusion matrix
confusion_matrix(y_test, y_pred_class)

array([[1203,    5],
       [  11,  174]], dtype=int64)

In [19]:
nb.predict_proba(X_test_dtm)

array([[9.97122551e-01, 2.87744864e-03],
       [9.99981651e-01, 1.83488846e-05],
       [9.97926987e-01, 2.07301295e-03],
       ...,
       [9.99998910e-01, 1.09026171e-06],
       [1.86697467e-10, 1.00000000e+00],
       [9.99999996e-01, 3.98279868e-09]])

The column on the left represents the probability the message is of class 0, and the column on the right represents the probability the message is of class 1.

In [20]:
# calculate predicted probabilities for X_test_dtm (poorly calibrated)
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([2.87744864e-03, 1.83488846e-05, 2.07301295e-03, ...,
       1.09026171e-06, 1.00000000e+00, 3.98279868e-09])

Understand that naive bayes predicts very extreme value and there probability provided by it should not be understood as probability in colloquial definition. It just performed some mathematical operation and gave out the numbers. In simple words, the predicted probability provided by naive bayes is not well caliberated.

In [21]:
# calculate AUC
roc_auc_score(y_test, y_pred_prob)

0.9866431000536962

In [22]:
X_dtm = cv.fit_transform(X)
nb.fit(X_dtm, y)

MultinomialNB()

In [24]:
import pickle
pickle.dump(cv, open("countvect.pkl", "wb"))
pickle.dump(nb, open("nb.pkl", "wb"))

### Logistic Regression

In [25]:
# import and instantiate a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [26]:
# train the model using X_train_dtm
logreg.fit(X_train_dtm, y_train)

LogisticRegression()

In [27]:
# make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test_dtm)

In [28]:
# calculate predicted probabilities for X_test_dtm (well calibrated)
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([0.00959377, 0.00295662, 0.00452424, ..., 0.031302  , 0.99748962,
       0.00119521])

The predicted probability provided by logistic regression is well caliberated.

In [29]:
# calculate accuracy
accuracy_score(y_test, y_pred_class)

0.9877961234745154

In [30]:
# calculate AUC
roc_auc_score(y_test, y_pred_prob)

0.9936280651512441

In [31]:
logreg.fit(X_dtm, y)

LogisticRegression()

In [32]:
pickle_out = open("logreg.pkl", "wb")
pickle.dump(logreg, pickle_out)